In [1]:
import numpy as np 
import pandas as pd
import json
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

## reading previously generated file

In [2]:
canada_df = pd.read_csv('postal_code_with_location.csv')
canada_df.head()

Postal Code      Borough                              Neighborhood  \
0         M1B  Scarborough                           Malvern , Rouge   
1         M1C  Scarborough  Rouge Hill , Port Union , Highland Creek   
2         M1E  Scarborough       Guildwood , Morningside , West Hill   
3         M1G  Scarborough                                    Woburn   
4         M1H  Scarborough                                 Cedarbrae   

    Latitude  Longitude  
0  43.806686 -79.194353  
1  43.784535 -79.160497  
2  43.763573 -79.188711  
3  43.770992 -79.216917  
4  43.773136 -79.239476

## only analyse toronto locations so only take data which contain "Toronto"

In [3]:
toronto_df = canada_df[canada_df['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_df.head()

Postal Code          Borough                     Neighborhood   Latitude  \
0         M4E     East Toronto                      The Beaches  43.676357   
1         M4K     East Toronto    The Danforth West , Riverdale  43.679557   
2         M4L     East Toronto  India Bazaar , The Beaches West  43.668999   
3         M4M     East Toronto                  Studio District  43.659526   
4         M4N  Central Toronto                    Lawrence Park  43.728020   

   Longitude  
0 -79.293031  
1 -79.352188  
2 -79.315572  
3 -79.340923  
4 -79.388790

## Now get geographical location of toronto using geopy library

In [4]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto canada are {}, {}.'.format(latitude, longitude))

C:\Users\user\Anaconda3\envs\tf_training\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of toronto canada are 43.6534817, -79.3839347.


## create map of Toronto using latitude and longitude values

In [5]:

toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

In [ ]:
CLIENT_ID = 'your id' # your Foursquare ID
CLIENT_SECRET = 'your key' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

### Now explore Neighborhood in Toronto

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [11]:
LIMIT = 100
radius = 500
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

The Beaches
The Danforth West , Riverdale
India Bazaar , The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park , Summerhill East
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
Rosedale
St. James Town , Cabbagetown
Church and Wellesley
Regent Park , Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond , Adelaide , King
Harbourfront East , Union Station , Toronto Islands
Toronto Dominion Centre , Design Exchange
Commerce Court , Victoria Hotel
Roselawn
Forest Hill North & West
The Annex , North Midtown , Yorkville
University of Toronto , Harbord
Kensington Market , Chinatown , Grange Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Stn A PO Boxes
First Canadian Place , Underground city
Christie
Dufferin , Dovercourt Village
Little Portugal , Trinity
Brockton , Parkdale Village , Exhibition Place
High Park , 

In [12]:
toronto_venues.head()

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  The Beaches              43.676357              -79.293031   
1  The Beaches              43.676357              -79.293031   
2  The Beaches              43.676357              -79.293031   
3  The Beaches              43.676357              -79.293031   
4  The Beaches              43.676357              -79.293031   

                                Venue  Venue Latitude  Venue Longitude  \
0                   Glen Manor Ravine       43.676821       -79.293942   
1  The Big Carrot Natural Food Market       43.678879       -79.297734   
2                 Grover Pub and Grub       43.679181       -79.297215   
3                   Glen Stewart Park       43.675278       -79.294647   
4                       Upper Beaches       43.680563       -79.292869   

      Venue Category  
0              Trail  
1  Health Food Store  
2                Pub  
3               Park  
4       Neighborhood

### Let's check how many venues were returned for each neighborhood

In [13]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            58   
Brockton , Parkdale Village , Exhibition Place                         24   
Business reply mail Processing CentrE                                  17   
CN Tower , King and Spadina , Railway Lands , H...                     16   
Central Bay Street                                                     64   
Christie                                                               17   
Church and Wellesley                                                   75   
Commerce Court , Victoria Hotel                                       100   
Davisville                                                             35   
Davisville North                                                        8   
Dufferin , Dovercourt Village                                          15   
First Canadian Place , Underground city                               100   
Forest Hill North & West                                                4   
Garden District, Ryerson                                              100   
Harbourfront East , Union Station , Toronto Isl...                    100   
High Park , The Junction South                                         22   
India Bazaar , The Beaches West                                        23   
Kensington Market , Chinatown , Grange Park                            57   
Lawrence Park                                                           3   
Little Portugal , Trinity                                              41   
Moore Park , Summerhill East                                            1   
North Toronto West                                                     22   
Parkdale , Roncesvalles                                                13   
Queen's Park , Ontario Provincial Government                           40   
Regent Park , Harbourfront                                             48   
Richmond , Adelaide , King                                             95   
Rosedale                                                                4   
Roselawn                                                                1   
Runnymede , Swansea                                                    39   
St. James Town                                                         81   
St. James Town , Cabbagetown                                           42   
Stn A PO Boxes                                                         97   
Studio District                                                        40   
Summerhill West , Rathnelly , South Hill , Fore...                     16   
The Annex , North Midtown , Yorkville                                  23   
The Beaches                                                             5   
The Danforth West , Riverdale                                          43   
Toronto Dominion Centre , Design Exchange                             100   
University of Toronto , Harbord                                        34   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             58   
Brockton , Parkdale Village , Exhibition Place                          24   
Business reply mail Processing CentrE                                   17   
CN Tower , King and Spadina , Railway Lands , H...                      16   
Central Bay Street                                                      64   
Christie                                                                17   
Church and Wellesley                                                    75   
Commerce Court , Victoria Hotel                                        100   
Davisville                                                              35   
Davisville North                    

In [14]:
print('numbers of unique catagories {}'.format(len(toronto_venues['Venue Category'].unique())))

numbers of unique catagories 226


### Analyse each Neighborhood

In [15]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Airport  Airport Food Court  Airport Gate  Airport Lounge  \
0            0        0                   0             0               0   
1            0        0                   0             0               0   
2            0        0                   0             0               0   
3            0        0                   0             0               0   
4            0        0                   0             0               0   

   Airport Service  Airport Terminal  American Restaurant  Antique Shop  \
0                0                 0                    0             0   
1                0                 0                    0             0   
2                0                 0                    0             0   
3                0                 0                    0             0   
4                0                 0                    0             0   

   Aquarium  ...  Theater  Theme Restaurant  Toy / Game Store  Trail  \
0         0  ...        0                 0                 0      1   
1         0  ...        0                 0                 0      0   
2         0  ...        0                 0                 0      0   
3         0  ...        0                 0                 0      0   
4         0  ...        0                 0                 0      0   

   Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0              0                              0                 0   
1              0                              0                 0   
2              0                              0                 0   
3              0                              0                 0   
4              0                              0                 0   

   Vietnamese Restaurant  Wine Bar  Women's Store  
0                      0         0              0  
1                      0         0              0  
2                      0         0              0  
3                      0         0              0  
4                      0         0              0  

[5 rows x 226 columns]

In [16]:
toronto_onehot.shape

(1623, 226)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [17]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

Neighborhood  Yoga Studio  Airport  \
0                                        Berczy Park     0.000000   0.0000   
1     Brockton , Parkdale Village , Exhibition Place     0.000000   0.0000   
2              Business reply mail Processing CentrE     0.058824   0.0000   
3  CN Tower , King and Spadina , Railway Lands , ...     0.000000   0.0625   
4                                 Central Bay Street     0.015625   0.0000   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0              0.0000        0.0000           0.000           0.0000   
1              0.0000        0.0000           0.000           0.0000   
2              0.0000        0.0000           0.000           0.0000   
3              0.0625        0.0625           0.125           0.1875   
4              0.0000        0.0000           0.000           0.0000   

   Airport Terminal  American Restaurant  Antique Shop  ...  Theater  \
0             0.000                  0.0           0.0  ...      0.0   
1             0.000                  0.0           0.0  ...      0.0   
2             0.000                  0.0           0.0  ...      0.0   
3             0.125                  0.0           0.0  ...      0.0   
4             0.000                  0.0           0.0  ...      0.0   

   Theme Restaurant  Toy / Game Store  Trail  Train Station  \
0               0.0               0.0    0.0            0.0   
1               0.0               0.0    0.0            0.0   
2               0.0               0.0    0.0            0.0   
3               0.0               0.0    0.0            0.0   
4               0.0               0.0    0.0            0.0   

   Vegetarian / Vegan Restaurant  Video Game Store  Vietnamese Restaurant  \
0                       0.017241               0.0                    0.0   
1                       0.000000               0.0                    0.0   
2                       0.000000               0.0                    0.0   
3                       0.000000               0.0                    0.0   
4                       0.015625               0.0                    0.0   

   Wine Bar  Women's Store  
0       0.0            0.0  
1       0.0            0.0  
2       0.0            0.0  
3       0.0            0.0  
4       0.0            0.0  

[5 rows x 226 columns]

## Let's get top venues and  put that into a pandas dataframe

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1     Brockton , Parkdale Village , Exhibition Place                  Café   
2              Business reply mail Processing CentrE    Light Rail Station   
3  CN Tower , King and Spadina , Railway Lands , ...       Airport Service   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0          Cocktail Bar                  Café              Beer Bar   
1        Breakfast Spot             Nightclub           Coffee Shop   
2                  Park        Farmers Market                   Spa   
3      Airport Terminal        Airport Lounge           Coffee Shop   
4    Italian Restaurant                  Café        Sandwich Place   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0         Farmers Market    Seafood Restaurant                Bakery   
1  Performing Arts Venue            Restaurant    Italian Restaurant   
2          Auto Workshop           Pizza Place            Restaurant   
3        Harbor / Marina              Boutique      Sculpture Garden   
4         Ice Cream Shop           Salad Place       Bubble Tea Shop   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0            Restaurant           Cheese Shop     Italian Restaurant  
1         Burrito Place          Intersection                    Bar  
2            Smoke Shop         Burrito Place            Yoga Studio  
3         Boat or Ferry          Airport Gate     Airport Food Court  
4          Burger Joint   Fried Chicken Joint         Discount Store

## Cluster Neighborhoods

In [20]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [21]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

Postal Code          Borough                     Neighborhood   Latitude  \
0         M4E     East Toronto                      The Beaches  43.676357   
1         M4K     East Toronto    The Danforth West , Riverdale  43.679557   
2         M4L     East Toronto  India Bazaar , The Beaches West  43.668999   
3         M4M     East Toronto                  Studio District  43.659526   
4         M4N  Central Toronto                    Lawrence Park  43.728020   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.293031               0                   Pub                  Park   
1 -79.352188               0      Greek Restaurant    Italian Restaurant   
2 -79.315572               0        Sandwich Place  Fast Food Restaurant   
3 -79.340923               0                  Café           Coffee Shop   
4 -79.388790               0           Swim School                  Park   

  3rd Most Common Venue   4th Most Common Venue 5th Most Common Venue  \
0                 Trail       Health Food Store      Cuban Restaurant   
1           Coffee Shop  Furniture / Home Store             Bookstore   
2           Pizza Place               Pet Store               Brewery   
3             Gastropub                 Brewery                Bakery   
4              Bus Line           Women's Store            Donut Shop   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0            Donut Shop      Doner Restaurant               Dog Run   
1        Ice Cream Shop           Yoga Studio           Pizza Place   
2    Italian Restaurant         Movie Theater            Restaurant   
3   American Restaurant           Yoga Studio     Convenience Store   
4      Doner Restaurant               Dog Run   Distribution Center   

  9th Most Common Venue 10th Most Common Venue  
0   Distribution Center         Discount Store  
1          Liquor Store                    Spa  
2         Burrito Place         Ice Cream Shop  
3        Sandwich Place            Cheese Shop  
4        Discount Store                  Diner

Finally, let's visualize the resulting clusters

In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters